In [65]:
import os
from PIL import Image
import json
import math

In [66]:
TRAIN = True
VAL = False
TEST = False

In [67]:
# only one can be true at a time, check and break if more than one is true
if TRAIN and VAL:
    print("Error: Both TRAIN and VAL are set to True. Only one can be True at a time.")
    exit(1)
elif TRAIN and TEST:
    print("Error: Both TRAIN and TEST are set to True. Only one can be True at a time.")
    exit(1)
elif VAL and TEST:
    print("Error: Both VAL and TEST are set to True. Only one can be True at a time.")
    exit(1)

In [68]:
if TRAIN:
    path = 'train_s'
    imagen = 'p106.jpg'

if VAL:
    path = 'val'
    imagen = 'p1428.jpg'

if TEST:
    path = 'test'
    imagen = 'p103.jpg'

In [69]:
# Define paths
# Directory with the YOLO annotations
annotation_directory = f'/path/to/data/SBH2024/good_data/data/labels/{path}/'
# Directory with the corresponding test images
image_directory = f'/path/to/data/SBH2024/good_data/data/images/{
    path}/'
# Directory to save the measurement features
output_directory = f'/path/to/data/SBH2024/good_data/data/measurements/{
    path}/'

In [70]:
# Function to parse a YOLO annotation file
def parse_yolo_annotation(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    boxes = []
    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center, y_center, width, height = map(float, parts[1:])
        boxes.append((class_id, x_center, y_center, width, height))
    return boxes

In [71]:
# Load all annotations
annotations = {}
for file_name in os.listdir(annotation_directory):
    if file_name.endswith('.txt'):
        file_path = os.path.join(annotation_directory, file_name)
        annotations[file_name] = parse_yolo_annotation(file_path)

print("Annotations loaded successfully.")

Annotations loaded successfully.


In [72]:
# Function to calculate the diagonal length of a bounding box
def calculate_diagonal_length(width, height):
    return math.sqrt(width**2 + height**2)

In [73]:
# Function to calculate pixel-to-mm conversion using the diagonal of the bounding box
def calculate_pixel_to_mm(image_height, image_width, annotation):
    stick_length_mm = 500  # The actual length of the measurement stick in millimeters
    for box in annotation:
        class_id, _, _, width, height = box
        if class_id in [1, 2]:  # Measurement stick class
            # Calculate the diagonal length in pixels
            diagonal_length_px = calculate_diagonal_length(
                width * image_width, height * image_height)

            # Calculate the conversion factor from pixels to millimeters
            return stick_length_mm / diagonal_length_px
    return None

In [74]:
# Example: Calculate pixel-to-mm for a single image
image_name = imagen
annotation_file_name = image_name.replace('.jpg', '.txt')

# Load the image to get its dimensions
image_path = os.path.join(image_directory, image_name)
image = Image.open(image_path)
image_width, image_height = image.size

# Get the annotation for this image
annotation = annotations[annotation_file_name]

# Calculate the pixel-to-mm conversion factor
pixel_to_mm = calculate_pixel_to_mm(image_height, image_width, annotation)
print(f"Pixel-to-MM conversion factor for {image_name}: {pixel_to_mm} mm/px")

Pixel-to-MM conversion factor for p106.jpg: 1.7080997514721032 mm/px


In [75]:
# Function to estimate pothole size in millimeters
def estimate_pothole_size_mm(pixel_to_mm, annotation, image_width, image_height):
    pothole_size_mm = {}
    # if 0 not in classs_id, return 0 for all values
    if 0 not in [box[0] for box in annotation]:
        pothole_size_mm['width_mm'] = 500
        pothole_size_mm['height_mm'] = 500
        pothole_size_mm['area_mm2'] = 250000
        pothole_size_mm['aspect_ratio'] = 1
    for box in annotation:
        class_id, x_center, y_center, width, height = box
        if class_id == 0:  # Pothole class
            pothole_width_px = width * image_width
            pothole_height_px = height * image_height
            pothole_width_mm = pothole_width_px * pixel_to_mm
            pothole_height_mm = pothole_height_px * pixel_to_mm
            pothole_size_mm['width_mm'] = pothole_width_mm
            pothole_size_mm['height_mm'] = pothole_height_mm
            pothole_size_mm['area_mm2'] = pothole_width_mm * pothole_height_mm
            pothole_size_mm['aspect_ratio'] = pothole_width_mm / \
                pothole_height_mm
    return pothole_size_mm

In [76]:
# Estimate the pothole size in millimeters for this example image
pothole_size_mm = estimate_pothole_size_mm(
    pixel_to_mm, annotation, image_width, image_height)
print(f"Pothole dimensions for {image_name}: {pothole_size_mm}")

Pothole dimensions for p106.jpg: {'width_mm': 664.4951353435978, 'height_mm': 494.69244028462873, 'area_mm2': 328720.720060389, 'aspect_ratio': 1.3432490194539268}


In [77]:
results = {}
skipped = 0

for image_name in os.listdir(image_directory):
    if image_name.endswith('.jpg'):  # Adjust if you have .jpg or other formats
        annotation_file_name = image_name.replace('.jpg', '.txt')

        # Load the image
        image_path = os.path.join(image_directory, image_name)
        image = Image.open(image_path)
        image_width, image_height = image.size

        # Default values
        results[image_name] = {
            'width_mm': 0,
            'height_mm': 0,
            'area_mm2': 0,
            'aspect_ratio': 0
        }

        # Get the annotation
        annotation = annotations.get(annotation_file_name)
        if not annotation:
            continue  # Skip if no annotation found

        # Calculate pixel-to-mm conversion
        pixel_to_mm = calculate_pixel_to_mm(
            image_height, image_width, annotation)
        if not pixel_to_mm:
            print(f"No measurement stick found for {image_name}. Skipping...")
            skipped += 1
            continue

        # Estimate pothole size
        pothole_size_mm = estimate_pothole_size_mm(
            pixel_to_mm, annotation, image_width, image_height)
        results[image_name] = pothole_size_mm

print("All images processed. Pothole dimensions calculated.")
print(f"Skipped {skipped} images without measurement stick.")

No measurement stick found for p101.jpg. Skipping...
No measurement stick found for p102.jpg. Skipping...
No measurement stick found for p1037.jpg. Skipping...
No measurement stick found for p1055.jpg. Skipping...
No measurement stick found for p1078.jpg. Skipping...
No measurement stick found for p1082.jpg. Skipping...
No measurement stick found for p1089.jpg. Skipping...
No measurement stick found for p1095.jpg. Skipping...
No measurement stick found for p1103.jpg. Skipping...
No measurement stick found for p1104.jpg. Skipping...
No measurement stick found for p1113.jpg. Skipping...
No measurement stick found for p1117.jpg. Skipping...
No measurement stick found for p1123.jpg. Skipping...
No measurement stick found for p1131.jpg. Skipping...
No measurement stick found for p1143.jpg. Skipping...
No measurement stick found for p1149.jpg. Skipping...
No measurement stick found for p1154.jpg. Skipping...
No measurement stick found for p1156.jpg. Skipping...
No measurement stick found for

In [78]:
results[imagen]

{'width_mm': 664.4951353435978,
 'height_mm': 494.69244028462873,
 'area_mm2': 328720.720060389,
 'aspect_ratio': 1.3432490194539268}

In [79]:
len(results)

660

# Save the data

In [80]:
with open(output_directory + 'measurement_features.json', 'w') as f:
    json.dump(results, f)